# Setup GPU

In [1]:
from __future__ import absolute_import, division, print_function, unicode_literals

import tensorflow as tf

# 读取TF 配置, 防止出现BLAS错误

config = tf.compat.v1.ConfigProto(gpu_options = 
                         tf.compat.v1.GPUOptions(per_process_gpu_memory_fraction=0.8)
)
config.gpu_options.allow_growth = True
session = tf.compat.v1.Session(config=config)
tf.compat.v1.keras.backend.set_session(session)

# 显示变量的位置是否在GPU
tf.debugging.set_log_device_placement(True)

print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))

Num GPUs Available:  1


# 获取数据

In [2]:
import tensorflow as tf
from tensorflow import keras
(x_train, y_train), (x_val, y_val) = keras.datasets.fashion_mnist.load_data()

In [3]:
x_train.shape

(60000, 28, 28)

In [4]:
y_train.shape

(60000,)

# 预处理数据

In [5]:
def preprocess(x, y):
    x = tf.cast(x, tf.float32) / 255.0
    y = tf.cast(y, tf.int64)
    return x, y

In [6]:
 def create_dataset(xs, ys, n_classes=10):
        ys = tf.one_hot(ys, depth=n_classes)
        return tf.data.Dataset.from_tensor_slices((xs, ys)).map(preprocess).shuffle(len(ys)).batch(128)
?preprocess    

Signature: preprocess    (x, y)
Docstring: <no docstring>
File:      d:\bookreading\hackers_guide_to_machine_learning_with_python\<ipython-input-5-2d2d07ce58d4>
Type:      function


In [7]:
train_dataset = create_dataset(x_train, y_train)
train_dataset

Executing op OneHot in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op TensorSliceDataset in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op MapDataset in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op AnonymousRandomSeedGenerator in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op ShuffleDatasetV2 in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op BatchDatasetV2 in device /job:localhost/replica:0/task:0/device:CPU:0


<BatchDataset shapes: ((None, 28, 28), (None, 10)), types: (tf.float32, tf.int64)>

In [8]:
val_dataset = create_dataset(x_val, y_val)
val_dataset

Executing op MapDataset in device /job:localhost/replica:0/task:0/device:CPU:0


<BatchDataset shapes: ((None, 28, 28), (None, 10)), types: (tf.float32, tf.int64)>

# 建立模型

In [9]:
model = keras.Sequential([
    keras.layers.Reshape(
        target_shape=(28 * 28,), input_shape=(28, 28)
    ),
    keras.layers.Dense(
        units=256, activation='relu'
    ),
    keras.layers.Dense(
        units=192, activation='relu'
    ),
    keras.layers.Dense(
        units=128, activation='relu'),
    keras.layers.Dense(
        units=10, activation='softmax'
    )
])

Executing op RandomUniform in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op Sub in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op Mul in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op Add in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op VarHandleOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op VarIsInitializedOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op LogicalNot in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op Assert in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op AssignVariableOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op Fill in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op VarHandleOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op VarHandleOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op VarHandleOp in device /job:localhost/replica:0/tas

# 训练

In [10]:
model.compile(
    optimizer='adam',
    loss=tf.losses.CategoricalCrossentropy(from_logits=True),
    metrics=['accuracy']
)

history = model.fit(
    train_dataset.repeat(),
    epochs=10,
    steps_per_epoch=500,
    validation_data=val_dataset.repeat(),
    validation_steps=2
)

Executing op VarHandleOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op RepeatDataset in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op DatasetCardinality in device /job:localhost/replica:0/task:0/device:CPU:0
Train for 500 steps, validate for 2 steps
Epoch 1/10
Executing op OptimizeDataset in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op ModelDataset in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op AnonymousIteratorV2 in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op MakeIterator in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op VarHandleOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op AssignVariableOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op VarHandleOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op VarHandleOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op VarHandleOp in device /job

# Making predictions

In [13]:
predictions = model.predict(val_dataset)
predictions.shape

(10000, 10)

In [15]:
import numpy as np
np.argmax(predictions[0])

7